In [1]:
import glob
import os
import sys

try:
    sys.path.append(glob.glob('carla/PythonAPI/carla/dist/carla-*%d.%d-%s.egg' % (
        sys.version_info.major,
        sys.version_info.minor,
        'win-amd64' if os.name == 'nt' else 'linux-x86_64'))[0])
except IndexError:
    pass

import carla

import argparse
import math
import gin
gin.enter_interactive_mode()

import logging
from numpy import random
import time

from utils.weather import Weather

In [6]:
def get_actor_blueprints(world, filter, generation):
    bps = world.get_blueprint_library().filter(filter)

    if generation.lower() == "all":
        return bps

    # If the filter returns only one bp, we assume that this one needed
    # and therefore, we ignore the generation
    if len(bps) == 1:
        return bps

    try:
        int_generation = int(generation)
        # Check if generation is in available generations
        if int_generation in [1, 2]:
            bps = [x for x in bps if int(x.get_attribute('generation')) == int_generation]
            return bps
        else:
            print("   Warning! Actor Generation is not valid. No actor will be spawned.")
            return []
    except:
        print("   Warning! Actor Generation is not valid. No actor will be spawned.")
        return []

@gin.configurable
def main(
    host:str, port:int, number_of_vehicles: int, number_of_walkers: int, safe: bool, filterv: str, generationv: str, 
    filterw: str, generationw: str, tm_port:int, asynch:bool, hybrid: bool, seed: int, 
    seedw: int, car_lights_on: bool, hero: bool, respawn: bool, no_rendering: bool):

    

    logging.basicConfig(format='%(levelname)s: %(message)s', level=logging.INFO)

    vehicles_list = []
    walkers_list = []
    all_id = []
    client = carla.Client(host, port)
    client.set_timeout(10.0)
    synchronous_master = False
    random.seed(seed if seed is not None else int(time.time()))

    try:
        world = client.get_world()

        traffic_manager = client.get_trafficmanager(tm_port)
        traffic_manager.set_global_distance_to_leading_vehicle(2.5)
        if respawn:
            traffic_manager.set_respawn_dormant_vehicles(True)
        if hybrid:
            traffic_manager.set_hybrid_physics_mode(True)
            traffic_manager.set_hybrid_physics_radius(70.0)
        if seed is not None:
            traffic_manager.set_random_device_seed(seed)

        settings = world.get_settings()
        if not asynch:
            traffic_manager.set_synchronous_mode(True)
            if not settings.synchronous_mode:
                synchronous_master = True
                settings.synchronous_mode = True
                settings.fixed_delta_seconds = 0.05
            else:
                synchronous_master = False
        else:
            print("You are currently in asynchronous mode. If this is a traffic simulation, \
            you could experience some issues. If it's not working correctly, switch to synchronous \
            mode by using traffic_manager.set_synchronous_mode(True)")

        if no_rendering:
            settings.no_rendering_mode = True
        world.apply_settings(settings)

        blueprints = get_actor_blueprints(world, filterv, generationv)
        blueprintsWalkers = get_actor_blueprints(world, filterw, generationw)

        if safe:
            blueprints = [x for x in blueprints if int(x.get_attribute('number_of_wheels')) == 4]
            blueprints = [x for x in blueprints if not x.id.endswith('microlino')]
            blueprints = [x for x in blueprints if not x.id.endswith('carlacola')]
            blueprints = [x for x in blueprints if not x.id.endswith('cybertruck')]
            blueprints = [x for x in blueprints if not x.id.endswith('t2')]
            blueprints = [x for x in blueprints if not x.id.endswith('sprinter')]
            blueprints = [x for x in blueprints if not x.id.endswith('firetruck')]
            blueprints = [x for x in blueprints if not x.id.endswith('ambulance')]

        blueprints = sorted(blueprints, key=lambda bp: bp.id)

        spawn_points = world.get_map().get_spawn_points()
        number_of_spawn_points = len(spawn_points)

        if number_of_vehicles < number_of_spawn_points:
            random.shuffle(spawn_points)
        elif number_of_vehicles > number_of_spawn_points:
            msg = 'requested %d vehicles, but could only find %d spawn points'
            logging.warning(msg, number_of_vehicles, number_of_spawn_points)
            number_of_vehicles = number_of_spawn_points

        # @todo cannot import these directly.
        SpawnActor = carla.command.SpawnActor
        SetAutopilot = carla.command.SetAutopilot
        FutureActor = carla.command.FutureActor

        # --------------
        # Spawn vehicles
        # --------------
        batch = []
        hero = hero
        for n, transform in enumerate(spawn_points):
            if n >= number_of_vehicles:
                break
            blueprint = random.choice(blueprints)
            if blueprint.has_attribute('color'):
                color = random.choice(blueprint.get_attribute('color').recommended_values)
                blueprint.set_attribute('color', color)
            if blueprint.has_attribute('driver_id'):
                driver_id = random.choice(blueprint.get_attribute('driver_id').recommended_values)
                blueprint.set_attribute('driver_id', driver_id)
            if hero:
                blueprint.set_attribute('role_name', 'hero')
                hero = False
            else:
                blueprint.set_attribute('role_name', 'autopilot')

            # spawn the cars and set their autopilot and light state all together
            batch.append(SpawnActor(blueprint, transform)
                .then(SetAutopilot(FutureActor, True, traffic_manager.get_port())))

        for response in client.apply_batch_sync(batch, synchronous_master):
            if response.error:
                logging.error(response.error)
            else:
                vehicles_list.append(response.actor_id)

        # Set automatic vehicle lights update if specified
        if car_lights_on:
            all_vehicle_actors = world.get_actors(vehicles_list)
            for actor in all_vehicle_actors:
                traffic_manager.update_vehicle_lights(actor, True)

        # -------------
        # Spawn Walkers
        # -------------
        # some settings
        percentagePedestriansRunning = 0.0      # how many pedestrians will run
        percentagePedestriansCrossing = 0.0     # how many pedestrians will walk through the road
        if seedw:
            world.set_pedestrians_seed(seedw)
            random.seed(seedw)
        # 1. take all the random locations to spawn
        spawn_points = []
        for i in range(number_of_walkers):
            spawn_point = carla.Transform()
            loc = world.get_random_location_from_navigation()
            if (loc != None):
                spawn_point.location = loc
                spawn_points.append(spawn_point)
        # 2. we spawn the walker object
        batch = []
        walker_speed = []
        for spawn_point in spawn_points:
            walker_bp = random.choice(blueprintsWalkers)
            # set as not invincible
            if walker_bp.has_attribute('is_invincible'):
                walker_bp.set_attribute('is_invincible', 'false')
            # set the max speed
            if walker_bp.has_attribute('speed'):
                if (random.random() > percentagePedestriansRunning):
                    # walking
                    walker_speed.append(walker_bp.get_attribute('speed').recommended_values[1])
                else:
                    # running
                    walker_speed.append(walker_bp.get_attribute('speed').recommended_values[2])
            else:
                print("Walker has no speed")
                walker_speed.append(0.0)
            batch.append(SpawnActor(walker_bp, spawn_point))
        results = client.apply_batch_sync(batch, True)
        walker_speed2 = []
        for i in range(len(results)):
            if results[i].error:
                logging.error(results[i].error)
            else:
                walkers_list.append({"id": results[i].actor_id})
                walker_speed2.append(walker_speed[i])
        walker_speed = walker_speed2
        # 3. we spawn the walker controller
        batch = []
        walker_controller_bp = world.get_blueprint_library().find('controller.ai.walker')
        for i in range(len(walkers_list)):
            batch.append(SpawnActor(walker_controller_bp, carla.Transform(), walkers_list[i]["id"]))
        results = client.apply_batch_sync(batch, True)
        for i in range(len(results)):
            if results[i].error:
                logging.error(results[i].error)
            else:
                walkers_list[i]["con"] = results[i].actor_id
        # 4. we put together the walkers and controllers id to get the objects from their id
        for i in range(len(walkers_list)):
            all_id.append(walkers_list[i]["con"])
            all_id.append(walkers_list[i]["id"])
        all_actors = world.get_actors(all_id)

        # wait for a tick to ensure client receives the last transform of the walkers we have just created
        if asynch or not synchronous_master:
            world.wait_for_tick()
        else:
            world.tick()

        # 5. initialize each controller and set target to walk to (list is [controler, actor, controller, actor ...])
        # set how many pedestrians can cross the road
        world.set_pedestrians_cross_factor(percentagePedestriansCrossing)
        for i in range(0, len(all_id), 2):
            # start walker
            all_actors[i].start()
            # set walk to random point
            all_actors[i].go_to_location(world.get_random_location_from_navigation())
            # max speed
            all_actors[i].set_max_speed(float(walker_speed[int(i/2)]))

        print('spawned %d vehicles and %d walkers, press Ctrl+C to exit.' % (len(vehicles_list), len(walkers_list)))

        # Example of how to use Traffic Manager parameters
        traffic_manager.global_percentage_speed_difference(0.0)

        # Dynamic Weather
        speed_factor = 1.0
        update_freq = 0.5 / speed_factor


        weather = Weather(world.get_weather())

        elapsed_time = 0.0

        while True:
            if not asynch and synchronous_master:
                world.tick()
            else:
                # world.wait_for_tick()
                timestamp = world.wait_for_tick(seconds=30.0).timestamp
                elapsed_time += timestamp.delta_seconds
                if elapsed_time > update_freq:
                    weather.tick(speed_factor * elapsed_time)

                    # traffic_manager.global_percentage_speed_difference(30.0 - (10 * (weather._storm.rain / 100)))
                    world.set_weather(weather.weather)
                    sys.stdout.write('\r' + str(weather) + 12 * ' ')
                    # sys.stdout.write('\n' + str((30.0 - (10 * (weather._storm.rain / 100)))) + 12 * ' ')
                    # sys.stdout.write('\n' + str(world.get_snapshot().find(vehicles_list[10]).get_velocity()) + 12 * ' ')
                    # sys.stdout.write('\n' + str(world.get_snapshot().find(vehicles_list[10]).get_acceleration()) + 12 * ' ')
                    sys.stdout.flush()
                    elapsed_time = 0.0

    finally:

        if not asynch and synchronous_master:
            settings = world.get_settings()
            settings.synchronous_mode = False
            settings.no_rendering_mode = False
            settings.fixed_delta_seconds = None
            world.apply_settings(settings)

        print('\ndestroying %d vehicles' % len(vehicles_list))
        client.apply_batch([carla.command.DestroyActor(x) for x in vehicles_list])

        # stop walker controllers (list is [controller, actor, controller, actor ...])
        for i in range(0, len(all_id), 2):
            all_actors[i].stop()

        print('\ndestroying %d walkers' % len(walkers_list))
        client.apply_batch([carla.command.DestroyActor(x) for x in all_id])

        time.sleep(0.5)

In [7]:
gin.parse_config_file('configs/config.gin')
try:
    main()
except KeyboardInterrupt:
    pass
finally:
    print('\ndone.')

You are currently in asynchronous mode. If this is a traffic simulation,             you could experience some issues. If it's not working correctly, switch to synchronous             mode by using traffic_manager.set_synchronous_mode(True)


ERROR: Spawn failed because of collision at spawn position


spawned 50 vehicles and 9 walkers, press Ctrl+C to exit.
Sun(alt: -10.85, azm: 310.54) Storm(clouds=11%, rain=0%, wind=5%)            
destroying 50 vehicles

destroying 9 walkers

done.


In [25]:
speed_factor = 2.0
update_freq = 0.1 / speed_factor

client = carla.Client('127.0.0.1', 2000)
client.set_timeout(2.0)
world = client.get_world()

weather = Weather(world.get_weather())

elapsed_time = 0.0

while True:
    timestamp = world.wait_for_tick(seconds=30.0).timestamp
    elapsed_time += timestamp.delta_seconds
    if elapsed_time > update_freq:
        weather.tick(speed_factor * elapsed_time)
        world.set_weather(weather.weather)
        sys.stdout.write('\r' + str(weather) + 12 * ' ')
        sys.stdout.flush()
        elapsed_time = 0.0

Sun(alt: 46.79, azm: 15.15) Storm(clouds=0%, rain=0%, wind=5%)                 

KeyboardInterrupt: 

In [4]:
spawn_points = world.get_map().get_spawn_points()
number_of_spawn_points = len(spawn_points)
number_of_spawn_points

155

In [7]:
weather = world.get_weather()

In [21]:
weather.precipitation = 50
weather.wetness = 30
world.set_weather(weather)
world.get_weather().precipitation

50.0